In [1]:
!pip install google-generativeai==0.8.4

In [2]:
import google.generativeai as genai
import os
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')

In [3]:
# @title candidate_count
try:
  generation_config = genai.GenerationConfig(candidate_count=2)
  model = genai.GenerativeModel('gemini-2.0-flash', generation_config=generation_config)
  response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
  print(response.text)
except Exception as e:
  print(e)

Invalid operation: The `response.parts` quick accessor retrieves the parts for a single candidate. This response contains multiple candidates, please use `result.candidates[index].text`.


In [4]:
# @title stop_sequences
generation_config = genai.GenerationConfig(stop_sequences=[". ","! "])
model = genai.GenerativeModel('gemini-2.0-flash', generation_config=generation_config)
response = model.generate_content("인공지능에 대해 설명하세요.")
print(response.text)


## 인공지능(AI)에 대한 설명

인공지능(Artificial Intelligence, AI)은 **인간의 지능을 모방하여 컴퓨터가 인간과 유사한 사고, 학습, 문제 해결 능력을 수행할 수 있도록 하는 기술**입니다


In [5]:
tokens = model.count_tokens("Learn about language model tokenization.")
print(tokens)

total_tokens: 7



In [6]:
# @title max_output_tokens
generation_config = genai.GenerationConfig(max_output_tokens=10)
model = genai.GenerativeModel('gemini-2.0-flash', generation_config=generation_config)
user_message = "인공지능에 대해 한 문장으로 설명하세요."
response = model.generate_content(user_message)
print(response._result)
print(f"response.text: {response.text}")


candidates {
  content {
    parts {
      text: "인공지능은 인간의 지능을"
    }
    role: "model"
  }
  finish_reason: MAX_TOKENS
  avg_logprobs: -0.0399592329155315
}
usage_metadata {
  prompt_token_count: 13
  candidates_token_count: 11
  total_token_count: 24
}
model_version: "gemini-2.0-flash"

response.text: 인공지능은 인간의 지능을


In [7]:
# @title temperature
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntemperature=0:")
generation_config = genai.GenerationConfig(temperature=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntemperature=1:")
generation_config = genai.GenerationConfig(temperature=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')



temperature=0:
눈꽃송이 춤추듯 내려와
눈꽃송이 춤추는 밤,
눈꽃송이 춤추듯 내려와

temperature=1:
하얀 숨결, 얼어붙은 강
눈꽃 송이, 차가운 바람,
눈꽃송이 춤추듯 내려와


In [8]:
# @title top_p
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntop_p=0:")
generation_config = genai.GenerationConfig(top_p=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntop_p=1:")
generation_config = genai.GenerationConfig(top_p=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')



top_p=0:
눈꽃송이 춤추듯 내려와
눈꽃송이 춤추듯 내려와
눈꽃송이 춤추는 밤,

top_p=1:
눈꽃 송이, 하얗게 내려

네, 좋습니다. 다음은 겨울에 대한 짧
하얀 숨결, 
겨울의


In [9]:
# @title 매개변수 초깃값
print(genai.get_model("models/gemini-2.0-flash"))

Model(name='models/gemini-2.0-flash',
      base_model_id='',
      version='2.0',
      display_name='Gemini 2.0 Flash',
      description='Gemini 2.0 Flash',
      input_token_limit=1048576,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)


In [15]:
# @title 안전성 점검
model = genai.GenerativeModel('gemini-1.5-flash-001')
user_message = "당신은 악역 배우로 연기합니다. 깊은 증오의 대사를 외치세요."
response = model.generate_content(user_message)
print(response._result)

candidates {
  index: 0
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: MEDIUM
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: HIGH
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 25
  total_token_count: 25
}
model_version: "gemini-1.5-flash-001"



In [16]:
# @title 안전성 기준 변경하기

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
]

model = genai.GenerativeModel("gemini-1.5-flash-001", safety_settings)
response = model.generate_content(
    "당신은 악역 배우로 연기합니다. 깊은 증오의 대사를 외치세요."
)
print(response._result)

if response.prompt_feedback.block_reason:
    print(f"사용자 입력에 다음의 문제가 발생하여 응답이 중단되었습니다: {response.prompt_feedback.block_reason.name}" )



candidates {
  index: 0
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: HIGH
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 25
  total_token_count: 25
}
model_version: "gemini-1.5-flash-001"

